<a href="https://colab.research.google.com/github/maratsmuk/daily_dashboards/blob/main/%5Bdaily_dashboards%5Dinteractive_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Daily dashboards
## Created by M.S. Mukhametzhanov
### An interactive public dashboard is created on datapane.com: https://datapane.com/u/maratsmuk/reports/VkGygN3/daily-dashboards/
### Input: database with the following tables: 
1. Accounts(id UNSIGNED INT PK, open_dt DATE NOT NULL, close_dt DATE NOT NULL)
2. Dates(dt DATE PK)
3. Transactions(id UNSIGNED INT PK, account_id UNSIGNED INT NOT NULL, dttm DATETIME NOT NULL, amt FLOAT)


Connection to the database:

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls drive/MyDrive/daily_dashboards
mydb = 'drive/MyDrive/daily_dashboards/DB_accounts.db'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 accounts.csv					  DB_accounts.db
'[daily_dashboards]db_generation.ipynb'		  psw_datapane.txt
'[daily_dashboards]interactive_dashboard.ipynb'   transactions.csv
 dates.csv


Importing necessary libraries: 
- sqlite3 for accessing the database
- pandas for working with the resulting dataframes
- datetime, numpy for working with the respective standard data types
- datapane is the main library for creating the interactive dashboard
- matplotlib and plotly for creating the graphs
- json for connecting to the datapane account

In [32]:
import sqlite3
import pandas as pd
import datetime
import numpy as np
try:
  import datapane as dp
except:
  !pip install datapane
  import datapane as dp
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import json

Connect to the account on https://datapane.com/:

In [33]:
with open('drive/MyDrive/daily_dashboards/psw_datapane.txt','r') as f:
    data = f.read()
    js = json.loads(data)
    dp.login(js['token'])

Connected successfully to https://datapane.com as maratsmuk


Test the connection to the database and check its format (correct the queries if it is necessary, e.g., if the format of the data is different or ids are not integer):

In [34]:
db_connection = sqlite3.connect(mydb)
cursor = db_connection.cursor()
cursor.execute('''select * from accounts limit 5''')
print(cursor.fetchall())
cursor.execute('''select * from dates limit 5''')
print(cursor.fetchall())
cursor.execute('''select * from transactions limit 5''')
print(cursor.fetchall())
cursor.close()
db_connection.close()


[(1, '2015-03-16', '2016-10-05'), (2, '2021-05-16', '2021-08-19'), (3, '2017-07-22', '2018-09-30'), (4, '2013-08-25', '2019-10-31'), (5, '2015-01-21', '2016-05-11')]
[('2010-01-01',), ('2010-01-02',), ('2010-01-03',), ('2010-01-04',), ('2010-01-05',)]
[(1, 13, '2010-02-02 00:00:00', 0.0), (2, 23, '2010-02-07 00:00:00', 0.0), (3, 33, '2010-02-23 00:00:00', 0.0), (4, 13, '2010-02-25 18:49:07', 116.23), (5, 33, '2010-02-26 12:29:28', -805.5)]


Main functions for creating the resulting dashboards as pandas dataframes: 
- generate_account_history reads the data from the database
- generate_dashboard_account creates the pandas dataframe of daily balances for each account
- generate_plot and generate_plot_go create the graph objects for the plots (the latter is used)


In [35]:
def generate_account_history(acc_id,con):
  query = '''SELECT date(dttm) as dt, sum(amt) as amt from (SELECT * from transactions where account_id == {acc}) as table_acc group by dt'''.format(acc=acc_id)
  df_account = pd.read_sql_query(query,con)
  cursor = con.cursor()
  cursor.execute('''select open_dt,close_dt from accounts where id=={acc}'''.format(acc=acc_id))
  dates_account = cursor.fetchall()
  cursor.close()
  return df_account,dates_account
def generate_dashboard_account(acc_id,con):
  df_acc,dates_acc = generate_account_history(acc_id,con)
  datenow = datetime.date.fromisoformat(dates_acc[0][1])
  datenow = min(datenow,datetime.date.today()-datetime.timedelta(days=1))
  start = datetime.date.fromisoformat(dates_acc[0][0])
  account_history = pd.DataFrame(columns = ['dates','balance'])
  account_history['dates'] = [start + datetime.timedelta(days=i) for i in range((datenow-start).days+1)]
  currency_account = 0
  idx_df = 0
  for idx in account_history.index:
    if idx_df<=df_acc.index.max() and datetime.date.fromisoformat(df_acc.iloc[idx_df,0])==account_history.iloc[idx,0]:
      currency_account+= df_acc.iloc[idx_df,1] 
      idx_df+=1
    account_history.at[idx,'balance'] = currency_account
  return account_history
def generate_plot(acc_id):
  fig, ax = plt.subplots(figsize=(30,15))
  ax.set_title('Balance: account '+str(acc_id),fontsize=30)
  ax.plot(main_dash[acc_id]['dates'],main_dash[acc_id]['balance'],'r')
  ax.tick_params(axis='x', which='both',labelsize = 20,rotation=90)
  ax.tick_params(axis='y', which='major',labelsize = 20,rotation=0)
  ax.legend(['Account '+str(acc_id)],fontsize=30)
  ax.grid(True,which='both',axis='both')
  ax.set_xlim([main_dash[acc_id]['dates'].min(),main_dash[acc_id]['dates'].max()])
  start_date = main_dash[acc_id]['dates'][0]
  xticks_values = []
  if start_date.day>1:
    xticks_values.append(start_date)
  for dt in main_dash[acc_id]['dates']:
    if dt.day==1:
      xticks_values.append(dt)
  end_date = main_dash[acc_id].iloc[-1,0]
  if end_date.day>1:
    xticks_values.append(end_date)
  ax.set_xticks(xticks_values)
  return fig
def generate_plot_go(acc_id):
  trace0 = go.Scatter(x=main_dash[acc_id]['dates'], y=main_dash[acc_id]['balance'], name='Balance: Account '+str(acc_id))
  fig0 = go.Figure([trace0])
  fig0.update_layout(
      title={
          'text': 'Daily balance: Account '+str(acc_id),
          'x':0.5,
          'xanchor': 'center'})
  return fig0

Connect to the database, read the data and create the dashboard tables:

In [36]:
db_connection = sqlite3.connect(mydb)
cursor = db_connection.cursor()
cursor.execute('''select distinct id from accounts''')
account_ids = list(map(lambda t: t[0], cursor.fetchall()))
cursor.close()
main_dash = {}
for acc_id in account_ids:
  account_history = generate_dashboard_account(acc_id,db_connection)
  main_dash[acc_id] = account_history
db_connection.close()

Create auxiliary dataframes reading the data from the database (only for visualization purposes: these data are not included in the daily balances, but it can be useful to show them on the dashboard):

In [37]:
db_connection = sqlite3.connect(mydb)
tables_transactions = {}
for acc_id in main_dash:
  tables_transactions[acc_id]=pd.read_sql_query('''SELECT * from transactions where account_id == {acc}'''.format(acc = acc_id),db_connection)
df_accounts = pd.read_sql_query('''SELECT * from accounts''',db_connection)
df_dates = pd.read_sql_query('''SELECT * from dates''',db_connection)
df_transactions = pd.read_sql_query('''SELECT * from transactions''',db_connection)
db_connection.close()
#df_accounts.set_index('id',inplace=True)
#df_dates.set_index('dt',inplace=True)
#df_transactions.set_index('id',inplace=True)

Create the interactive dashboard using the tables constructed so far (auxiliary functions check_closed and build_table_list are used only to improve the readability of the code). The transactions data is limited to the last 100 transactions only to guarantee the stability of the dashboard in the case of a large dataset (but the daily balances dashboard is created for the whole dataset).


In [53]:
def check_closed(acc_id):
  return 'closed' if datetime.date.fromisoformat(df_accounts[df_accounts.id==acc_id].iloc[0,2])<=datetime.date.today()-datetime.timedelta(days=1) else 'open'
def build_table_list(acc_id):
  df1 = tables_transactions[acc_id][tables_transactions[acc_id]['amt']>=0].tail(100)
  df2 = tables_transactions[acc_id][tables_transactions[acc_id]['amt']<=0].tail(100)
  df3 = tables_transactions[acc_id].tail(100)
  return ([dp.DataTable(df1,label='Incomes',caption='The last 100 transactions executed on Account '+str(acc_id)),
           dp.DataTable(df2,label='Outcomes',caption='The last 100 transactions executed on Account '+str(acc_id)),
           dp.DataTable(df3,label='Incomes and Outcomes',caption='The last 100 transactions executed on Account '+str(acc_id))]) 
account_list = list(main_dash.keys())
plt.figure(figsize=(10, 5), dpi=300)
plot_list = ([dp.Group(blocks=['### Open date: '+df_accounts[df_accounts.id==acc_id].iloc[0,1] + ', Close date: '+df_accounts[df_accounts.id==acc_id].iloc[0,2]+', State: '+check_closed(acc_id),
                               '### Initial balance: '+ str(round(main_dash[acc_id].iloc[0,1],2)) + ', Final balance: '+str(round(main_dash[acc_id].iloc[main_dash[acc_id].index[-1],1],2)),                             
                               dp.Plot(generate_plot_go(acc_id), label='Account '+str(acc_id))],columns=1,label='Account '+str(acc_id)) for acc_id in account_list])

table_list = ([dp.Group(blocks=["### Daily balances for Account "+str(acc_id), 
                                '### Open date: '+df_accounts[df_accounts.id==acc_id].iloc[0,1] + ', Close date: '+df_accounts[df_accounts.id==acc_id].iloc[0,2]+', State: '+check_closed(acc_id),
                                '### Initial balance: '+ str(round(main_dash[acc_id].iloc[0,1],2)) + ', Final balance: '+str(round(main_dash[acc_id].iloc[main_dash[acc_id].index[-1],1],2)),
                                dp.Group(blocks=[dp.Group(blocks=['###   ','###   ','###   ',dp.DataTable(main_dash[acc_id], caption='Daily balances for Account '+str(acc_id),label='Account '+str(acc_id))]),
                                                 dp.Select(blocks=build_table_list(acc_id))],columns=2,label='Account '+str(acc_id) )],
                        columns=1,label='Account '+str(acc_id))  for acc_id in account_list])        
g1 = dp.Group(blocks = ['### Accounts table', df_accounts.tail(100)],columns=1,label='Accounts')
g2 = dp.Group(blocks = ['### Dates table',df_dates.tail(100)],columns=1,label='Dates')
g3 = dp.Group(blocks = ['### Transactions table',df_transactions.tail(100)],columns=1,label='Transactions')
total_table_list = ([g1,g2,g3])        
tables = dp.Select(blocks = table_list)
plots = dp.Select(blocks = plot_list)
total_tables = dp.Select(blocks = total_table_list)

dp.Report(dp.Page(
    title="Tables Dashboard",
    blocks=["### Table Views", tables]
  ),
  dp.Page(
    title="Plot Dashboard",
    blocks=["### Graphical Dashboard", plots]
  ),
  dp.Page(
    title="Full Tables",
    blocks=["### Original database (only the last 100 records are displayed)", total_tables]
  )
  ).upload(name='Daily dashboards', description='Daily dashboards for each account',formatting=dp.ReportFormatting(width=dp.ReportWidth.FULL))

Uploading report and associated data - *please wait...*
Report successfully uploaded at https://datapane.com/u/maratsmuk/reports/VkGygN3/daily-dashboards/, follow the link to view and share your report.


<Figure size 3000x1500 with 0 Axes>